In [1]:
import requests
import json
import numpy as np
import pandas as pd

## The project's credentials 

To use the Spotify API, you need to have a Spotify account and create a new Web API project on the Spotify API website. Afterwards, you will be provided a Client ID and a Client Secret. Every project has different Client IDs and Secrets, even if created by the same user.

In [61]:
client_id = "be7f72bc052b4a9d9430b069de69a09f"
client_secret = "6a92bed59ca6490bb5b04d4393220b1f"

## Generate access token

In order to gain access to Spotify data, we need an access token. This is obtained through the POST request.

Afterwards, we are going to get the IDs of the artists from a given playlist (in our case, we selected "Liked Songs", one of my public Spotify playlists).

In [62]:
params = {"grant_type": "client_credentials", "client_id": client_id, "client_secret": client_secret}
header = {"Content-Type": "application/x-www-form-urlencoded"}
x = requests.post(r"https://accounts.spotify.com/api/token", data=params)

my_cool_data = json.loads(x.text)
access_token = my_cool_data['access_token']

playlist_id = "0NyzB57hbbDx5Enlkhw7Iv"
header =  {"Authorization": "Bearer {}".format(access_token)}
# params = {"access_token": access_token}

x = requests.get(r"https://api.spotify.com/v1/playlists/{}".format(playlist_id), headers=header)

x

<Response [200]>

**Response 200**: everything is okay

**Response 429**: too many requests made and you are banned for 13 hours or you need to create a new app and get new Cliend Id and Secret

Below, there is an example of artist information for a track.

In [4]:
json.loads(x.text)['tracks']['items'][10]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/5HFkc3t0HYETL4JeEbDB1v'},
  'href': 'https://api.spotify.com/v1/artists/5HFkc3t0HYETL4JeEbDB1v',
  'id': '5HFkc3t0HYETL4JeEbDB1v',
  'name': 'Powerwolf',
  'type': 'artist',
  'uri': 'spotify:artist:5HFkc3t0HYETL4JeEbDB1v'}]

## IDs of the Artists in the Playlist

We are going to collect the IDs of the artists that have songs in the specified playlist and add them to a list

In [5]:
artists_ids = []
artist_names = []
dictForClarity = {}

for elem in json.loads(x.text)['tracks']['items']:
    id = elem['track']['artists'][0]['id']
    if id not in artists_ids:
        artists_ids.append(id)
        artist_name = elem['track']['artists'][0]['name']
        artist_names.append(artist_name)
        dictForClarity[id] = artist_name
    else:
        continue

# artists_ids

# artists_id_np = np.array(artists_ids)
# len(json.loads(x.text)['tracks']['items']), len(artists_ids), len(np.unique(artists_id_np))

Next, we are going to get the IDs of all the albums every artist in our list has and then collect all the tracks and their collaborating artists from every album

In [6]:
def extract_albums_from_artists(artists_ids):
    """
        Extracts the albums for every artist having their ID in the list given as input.
        :param artists_ids: the list of the IDs of the artists
        :return: A list containing the IDs of all the albums belonging to the artist having the specified ID
    """
    albums_ids = []
    ct = 0
    for elem1 in artists_ids:
        while True:
            albumsRequest = requests.get(r"https://api.spotify.com/v1/artists/{}/albums".format(elem1), headers=header)
            if albumsRequest.status_code != 200:
                print("error", albumsRequest.status_code)
                if albumsRequest.status_code == 429:
                    print("Too many requests for the API! Stopping the process...")
                    break
                else:
                    continue
            break
        ct += 1
        # print(ct)
        for elem2 in json.loads(albumsRequest.text)["items"]:
            albums_ids.append(elem2['id'])

    return albums_ids

We are going to store in separate lists the following information:
- **tracks_ids**: the IDs of the tracks in an album
- **tracks_names**: the names of all the songs in an album
- **artists_pair_names**: pairs consisting of the names of the artists that collaborated on a song (if multiple artists are listed for a song, there will be n! pairs  of artists such that every artist from the collaborators list is paired with every other artist, but not with themselves)
- **artists_pair_ids**: pairs consisting of the IDs of the artists that collaborated on a song, following the same logic as *artists_pair_names*
- **albums_ids**: the IDs of the albums that we are going to use to extract the artists

In [12]:
tracks_ids = []
tracks_names = []
artists_pair_names = []
artists_pair_ids = []
albums_ids = []

In [13]:
albums_ids_observed = []
artists_left = artists_ids

In [18]:
def compare_two_pairs(pair1, pair2):
    """
        Check if there are any pairs formed by the same artist repeated twice.
    """
    return (pair1[0] == pair2[0] and pair1[1] == pair2[1]) or (pair1[0] == pair2[1] and pair1[0] == pair2[1])

def pair_in_list(pair, list):
    """
        Check if a pair belongs to a list.
    """
    for el in list:
        if compare_two_pairs(pair, el):
            return True

    return False 

while len(artists_left) != 0 or len(artists_ids) > 15000:
    print(len(artists_ids))
    if len(albums_ids) == 0:
        albums_ids = extract_albums_from_artists(artists_left)
        artists_left = []
    for elem1 in albums_ids:
        if elem1 not in albums_ids_observed:
            albums_ids_observed.append(elem1)
        else:
            continue
        while True:
            tracksAlbumsRequest = requests.get(r"https://api.spotify.com/v1/albums/{}/tracks".format(elem1), headers=header)
            if tracksAlbumsRequest.status_code != 200:
                print("error", tracksAlbumsRequest.status_code)
                continue
            break
        # print(tracksAlbumsRequest)
        for elem2 in json.loads(tracksAlbumsRequest.text)["items"]:
            n = len(elem2['artists'])
            if n > 1:
                for i in range(0, n-1):
                    artist_id1 = elem2['artists'][i]['id']
                    artist_name1 = elem2['artists'][i]['name']
                    if artist_id1 not in artists_ids:
                        artists_ids.append(artist_id1)
                        artists_left.append(artist_id1)
                    for j in range(1, n):
                        artist_id2 = elem2['artists'][j]['id']
                        artist_name2 = elem2['artists'][j]['name']
                        if not pair_in_list((artist_id1, artist_id2), artists_pair_ids):
                            artists_pair_ids.append((artist_id1, artist_id2))
                            artists_pair_names.append((artist_name1, artist_name2))
                            tracks_ids.append(elem2['id'])
                            tracks_names.append(elem2['name'])
                last_artist_id = elem2['artists'][n - 1]['id']
                if last_artist_id not in artists_ids:
                    artists_ids.append(last_artist_id)
                    artists_left.append(last_artist_id)
    albums_ids = []

5489
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429
error 429


KeyboardInterrupt: 

In [19]:
from collections import Counter

print("\nNumber of albums observed:", len(albums_ids_observed), 
      "\nNumber of albums unseen:", len(albums_ids),  
      "\nNumber of nodes unseen:", len(artists_left),
      "\nNumber of nodes:", len(artists_ids), 
      "\nNumber of edges:", len(artists_pair_ids),
      "\nNumber of unique nodes:", len(Counter(artists_ids).keys()))


Number of albums observed: 8758 
Number of albums unseen: 18183 
Number of nodes unseen: 2000 
Number of nodes: 14297 
Number of edges: 14019 
Number of unique nodes: 14297


We observed that the artist of the album can appear in the artists list as well. We will eliminate them using a for loop.

In [33]:
idx_to_be_removed = [i for i, el in enumerate(artists_pair_ids) if el[0] == el[1]]

artists_pair_ids_re = [el for i, el in enumerate(artists_pair_ids) if i not in idx_to_be_removed]
artists_pair_names_re = [el for i, el in enumerate(artists_pair_names) if i not in idx_to_be_removed]
tracks_ids_re = [el for i, el in enumerate(tracks_ids) if i not in idx_to_be_removed]
tracks_names_re = [el for i, el in enumerate(tracks_names) if i not in idx_to_be_removed]

len(artists_pair_ids_re), len(artists_pair_names_re)

(12178, 12178)

## Saving the data in CSV format

We will now begin to create the pieces of the CSV files: the nodes (the IDs of the artists) and the edges (the names of the songs, having as endpoints the pairs made out of the IDs of the artists that collaborated on the song, the names of the artists, the genres they appear most in and their popularity).

In [36]:
temp_artists_id = []
for elem in artists_pair_ids:
    if elem[0] not in temp_artists_id:
        temp_artists_id.append(elem[0])
    if elem[1] not in temp_artists_id:
        temp_artists_id.append(elem[1])


In [37]:
len(temp_artists_id), len(artists_ids)

(11488, 14295)

In [80]:
genres = []
popularity = []

In [81]:
start = 0
end = 50
step = 50
ct = 0
for i in range(len(temp_artists_id) // step):
    start = i * step
    end = (i + 1) * step
    ids = ",".join(temp_artists_id[start:end])
    features_request = requests.get(r"https://api.spotify.com/v1/artists?ids={}".format(ids), headers=header)
    
    text = json.loads(features_request.text)
    # print(text)
    for artist in text["artists"]:
        genres.append(artist['genres'])
        popularity.append(artist['popularity'])
    ct += 1
    print(ct)

if end < len(temp_artists_id):
    start = end
    end = len(temp_artists_id)
    ids = ",".join(temp_artists_id[start:end])
    features_request = requests.get(r"https://api.spotify.com/v1/artists?ids={}".format(ids), headers=header)
    
    text = json.loads(features_request.text)
    # print(text)
    for artist in text["artists"]:
        genres.append(artist['genres'])
        popularity.append(artist['popularity'])
    ct += 1
    print(ct)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230


In [82]:
len(genres), len(popularity), len(temp_artists_id)

(11488, 11488, 11488)

In [93]:
genre_list = []
default = "various"
for elem in genres:
    if len(elem) == 0:
        genre_list.append(default)
    else:
        genre_list.append(elem[0])
    



In [96]:
artists_ids_np = np.array(artists_pair_ids_re)
artists_names_np = np.array(artists_pair_names_re)
tracks_ids_np = np.array(tracks_ids_re)
tracks_names_np = np.array(tracks_names_re)

tracks_ids_npReshaped = tracks_ids_np.reshape(-1, 1)
tracks_names_npReshaped = tracks_names_np.reshape(-1, 1)
print(artists_ids_np.shape, tracks_ids_npReshaped.shape)
a = np.concatenate([artists_ids_np, artists_names_np, tracks_ids_npReshaped, tracks_names_npReshaped], axis=1)
a.shape

(12178, 2) (12178, 1)


(12178, 6)

In [97]:
data = pd.DataFrame(a, columns=["pair 1", "pair 2", "name 1", "name 2", "track id", "track name"])
data.to_csv('edges.csv', index=False)

In [110]:
genres_np = np.array(genre_list)
popularity_np = np.array(popularity)
genres_np = genres_np.reshape(-1, 1)
popularity_np = popularity_np.reshape(-1, 1)
artist_id_np = np.array(temp_artists_id).reshape(-1, 1)
nodes = np.concatenate([artist_id_np, genres_np, popularity_np], axis=1)
nodes.shape
#genres_np.shape, popularity_np.shape, artist_id_np.shape

(11488, 3)

In [111]:
dataNodes = pd.DataFrame(nodes, columns=["name", "genre", "popularity"])
dataNodes.to_csv('nodes.csv', index=False)

In [10]:
import pandas as pd

df1 = pd.read_csv("edges.csv")
df2 = pd.read_csv("nodes.csv")


In [11]:
dict = {}

for i in df1.index:
    dict[df1['pair 1'][i]] = df1['name 1'][i]
    dict[df1['pair 2'][i]] = df1['name 2'][i]

In [12]:
df2["name"] = None

In [13]:

for i in df2.index:
    if df2["ID"][i] in dict:
        df2["name"][i] = dict[df2["ID"][i]]
    # df1["popularity 2"][i] = dict[df1["pair 2"][i]]

C:\Users\Big Lion\AppData\Local\Temp\ipykernel_13340\1139956709.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["name"][i] = dict[df2["ID"][i]]


In [14]:
df2

,ID,genre,Label,name
0,2U2L1YxD0FCUkCdDfEbHm1,various,39,Joakim Brodén
1,0BsnL3av7z4uKVOX4QqTee,various,24,Chris Rörland
2,3o2dn2O0FCVsWDFSh8qxgG,power metal,69,Sabaton
3,4eBxTYQt9UEuAibNT1eJ0N,various,20,Jillian Aversa
4,4eiYHJLlxeKQXAw0UDoXJl,fantasy,43,Erutan
...,...,...,...,...
11483,50iN4QFD9Z9S1KAzDLvAju,anime rap,52,Divide Music
11484,5y9l3QufypVWqKc7TtLNfb,various,32,Villainous
11485,1LjfjX3LTdfwdkewSZkV1Z,various,3,None
11486,5iSgWCXiIYvctWEmKEP44X,various,19,None


In [15]:
df2.to_csv("another_merged.csv")